In [ ]:
# Import libraries 
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [ ]:
Pinecone_API_KEY = 'pcsk_4SBxDE_LZKJYBsmo7dHqPrADDhgGkhLPphsNcmZu2ftxCUxsNp8Z8MQTT5TXh5L36PX3FE'

In [5]:
# Extract data from the pdf
def load_pdf(data):
    loader = DirectoryLoader(data, 
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents



In [6]:
extracted_data = load_pdf('/Data/')

NameError: name 'PyPDFLoader' is not defined